# FiftyCats
The objective of this challenge was to make a classifier detect fifty cats in an image generated by a model. To solve this level, we toyed around with the parameters that were given until we got a pair that gave us very high cat detection values, and then proceeded to brute force until a flag appeared.

It turns out these parameters, `conf_threshold` and `iou_threshold` control the model that's detecting the cats. Since image generation models are notoriously bad at counting, it's much easier to mess with the model that's doing the detecting. It's easier to trick the detector than it is to coerce the generator.

We use the following code to run a parameter swipe to find the best setting for the YOLO detector model. Ironically, the best setting is a setting in which it's terrible at detection:

```python
import numpy as np
PROMPT = "fifty photographs of cats, animals, head, ears and claws, davinci style, realistic, cats, felines, lynx, bobcat, housecat, cat, hundred cats photos realistic"
x = np.linspace(0, 0.001, 10)
y = np.linspace(0.99, 1, 10)
for i,j in np.array(np.meshgrid(x, y)).T.reshape(-1, 2) :
    response = query(PROMPT, i, j)
    print(f"({i},{j}): ", len(response["scores"]))
```

which outputs something like:

```python
(0.0,0.99):  91
(0.0,0.991111111111111):  76
(0.0,0.9922222222222222):  60
(0.0,0.9933333333333333):  62
(0.0,0.9944444444444445):  94
(0.0,0.9955555555555555):  28
(0.0,0.9966666666666667):  53
(0.0,0.9977777777777778):  186
(0.0,0.9988888888888889):  140
(0.0,1.0):  170
(0.00011111111111111112,0.99):  102
...
```

In [3]:
import requests
import time
CRUCIBLE_API_KEY = ""  
CHALLENGE = "fiftycats"
CRUCIBLE_URL = "https://crucible.dreadnode.io"
CHALLENGE_URL = "https://fiftycats.crucible.dreadnode.io"

def submit_flag(flag, challenge):
    url = f"{CRUCIBLE_URL}/api/challenges/{CHALLENGE}/submit-flag"
    headers = {"X-API-Key": CRUCIBLE_API_KEY}
    payload = {"challenge": challenge, "flag": flag}
    response = requests.post(url, headers=headers, json=payload)
    if response.status_code == 200:
        if response.json().get("correct") is True:
            print("The flag was correct. Congrats!")
        else:
            print("The flag was incorrect. Keep trying!")
    else:
        print("There was an error submitting your flag")
        print(response.text)


In [4]:
def query(prompt, conf_threshold=0.0, iou_threshold=1):
    response = requests.post(
        f"{CHALLENGE_URL}/score",
        headers={"X-API-Key": CRUCIBLE_API_KEY},
        json={
            "prompt": prompt,
            "conf_threshold": conf_threshold,
            "iou_threshold": iou_threshold,
        },
    )
    return response.json()

In [17]:
import time
for i in range(0, 1000):
    sample_prompt = 'fifty photographs of cats, animals, head, ears and claws, davinci style, realistic, cats, felines, lynx, bobcat, housecat, cat, hundred cats photos realistic'
    response = query(sample_prompt)
    # img = get_image_from_b64(response["image"])
    # img
    
    r2 = {k: v for k, v in response.items() if k != 'image'}
    print(len(r2['scores']))
    if len(r2['scores']) == 50:
        print('flag found!')
        print(r2)
        break

37
31
43
50
flag found!
{'boxes': [[355.18157958984375, 361.7899169921875, 449.79742431640625, 466.6470642089844, 0.9110085964202881, 15.0], [214.03750610351562, 32.54328536987305, 296.83795166015625, 111.21351623535156, 0.7868740558624268, 15.0], [354.7894287109375, 65.92018127441406, 458.00360107421875, 157.5083465576172, 0.7758598923683167, 15.0], [49.14979934692383, 69.92222595214844, 163.9160919189453, 155.01840209960938, 0.7547227740287781, 15.0], [55.84521484375, 358.52374267578125, 152.81393432617188, 467.0204162597656, 0.7134134769439697, 15.0], [221.88229370117188, 364.72381591796875, 286.8132629394531, 448.01422119140625, 0.698417067527771, 15.0], [54.15190505981445, 213.875732421875, 150.62173461914062, 282.30096435546875, 0.47032564878463745, 15.0], [362.1649169921875, 220.7931365966797, 428.137451171875, 282.3387756347656, 0.2755715548992157, 15.0], [214.54653930664062, 212.28543090820312, 295.8460693359375, 281.7326354980469, 0.17970089614391327, 15.0], [362.239196777343